In [147]:
from astropop.math import QFloat
from astropy import units as u
from astropy import constants as const
import numpy as np

c = QFloat(const.c.value, None, const.c.unit)
G = QFloat(const.G.value, None, const.G.unit)

def format_to_nice_str(number, precision=None):
    value = number.nominal
    error = number.uncertainty
    unit = number.unit
    if precision is None:
        if error == 0:
            err_precision = 0
            val_precision = 3
        elif value == 0:
            val_precision = 0
            err_precision = 0
        else:
            value_exponent = int(np.log10(abs(value)))
            error_exponent = int(np.log10(abs(error)))
            precision = value_exponent - error_exponent
            val_precision = precision
            err_precision = precision
    return f"({value:.{val_precision}e} +- {error:.{err_precision}e}) {unit}"

# P5

#### Measured value for Einstein radius:

In [148]:
images = ("A", "B", "C", "D") # only used to print results

x_err = 2
y_err = 2

coords_A = (QFloat(1277.0, x_err, u.pix), QFloat(714.0, y_err, u.pix))
coords_B = (QFloat(1093.0, x_err, u.pix), QFloat(707.5, y_err, u.pix))
coords_C = (QFloat(1002.0, x_err, u.pix), QFloat(668.0, y_err, u.pix))
coords_D = (QFloat(1276.0, x_err, u.pix), QFloat(174.5, y_err, u.pix))

coords_all_images = [coords_A, coords_B, coords_C, coords_D]
coords_center = (QFloat(1206.1, 1, u.pix), QFloat(387.9, 1, u.pix))

conv_factor_px_arcsec = QFloat(0.1, None, u.arcsec / u.pixel)

def get_distance_in_arcsec(coords_a, coords_b):
    d_y = np.abs(coords_a[1] - coords_b[1])
    d_x = np.abs(coords_a[0] - coords_b[0])
    d_pixel = (np.sqrt(d_y**2 + d_x**2))
    print(f"Distance in pixel: {d_pixel}")
    d_arcsec = d_pixel * conv_factor_px_arcsec
    return d_arcsec

# calculate and print distances from image to center:
distance_to_center_all_images = []
print("\n") # create blank line
for pos, coords_image in enumerate(coords_all_images):
    distance = get_distance_in_arcsec(coords_image, coords_center)
    distance_to_center_all_images.append(distance)
    print(f"Distance from {images[pos]} to Center: {distance}", end="\n\n")

mean_distance_to_center = np.mean(distance_to_center_all_images)
mean_einstein_radius = mean_distance_to_center
print(f"Mean Einstein radius: {mean_einstein_radius}",
       end="\n\n")

# calculate and print half the distance from B to D:
einstein_radius_B_D = (get_distance_in_arcsec(coords_B, coords_D) / 2)
print(f"Einstein radius from distance B to D: {einstein_radius_B_D}")

# print deviation:
print(f"Deviation: einstein radius BD to mean einstein radius: {(100 * (1 - einstein_radius_B_D / mean_distance_to_center))}%")
print(f"Deviation: mean einstein radius to einstein radius BD: {(100 * (1 - mean_distance_to_center / einstein_radius_B_D))}%")



Distance in pixel: 334+-2 pix
Distance from A to Center: 33.4+-0.2 arcsec

Distance in pixel: 339+-2 pix
Distance from B to Center: 33.9+-0.2 arcsec

Distance in pixel: 347+-2 pix
Distance from C to Center: 34.7+-0.2 arcsec

Distance in pixel: 225+-2 pix
Distance from D to Center: 22.5+-0.2 arcsec

Mean Einstein radius: 31.1+-0.1 arcsec

Distance in pixel: 564+-3 pix
Einstein radius from distance B to D: 28.2+-0.1 arcsec
Deviation: einstein radius BD to mean einstein radius: 9.4+-0.6%
Deviation: mean einstein radius to einstein radius BD: -10.4+-0.7%


#### Velocity dispersion:

In [149]:
z_d = 0.39
u = 1/np.sqrt(1+z_d)

# Resetting the error on the einstein radius:

# error of 0.1 arcsec
einstein_radius_1 = QFloat(einstein_radius_B_D.nominal, 0.1, "arcsec")

# error of 1.0 arcsec
einstein_radius_2 = QFloat(einstein_radius_B_D.nominal, 1.0, "arcsec")

einstein_radius_combined = QFloat(29.7, 1.6, "arcsec")

def get_velocity_dispersion(einstein_radius, z):
    if z == "inf":
        velocity_dispersion = np.sqrt(einstein_radius.to("rad")*c**2/(4*np.pi*u))
    else:
        D_s_prop = (1+z)**(-1) - (1+z)**(-3/2) # prop means without constant
        D_ds_prop = u * (1+z)**(-1) - (1+z)**(-3/2)
        velocity_dispersion = np.sqrt(einstein_radius.to("rad") * c**2 / (4 * np.pi) * (D_s_prop / D_ds_prop))
    return velocity_dispersion.to("rad(1/2) km/s")


# For z = inf:
velocity_dispersion_inf_1 = get_velocity_dispersion(einstein_radius_1, z='inf')
velocity_dispersion_inf_2 = get_velocity_dispersion(einstein_radius_2, z='inf')

print(f"Velocity Dispersion for z -> inf, error 0.1: {format_to_nice_str(velocity_dispersion_inf_1)}")
print(f"Velocity Dispersion for z -> inf, error 1.0: {format_to_nice_str(velocity_dispersion_inf_2)}")

# For z = 1.6:
velocity_dispersion_1 = get_velocity_dispersion(einstein_radius_1, z=1.6)
velocity_dispersion_2 = get_velocity_dispersion(einstein_radius_2, z=1.6)
velocity_dispersion_combined = get_velocity_dispersion(einstein_radius_combined, z=1.6)

print(f"Velocity Dispersion for z = 1.6, error 0.1: {format_to_nice_str(velocity_dispersion_1)}")
print(f"Velocity Dispersion for z = 1.6, error 1.0: {format_to_nice_str(velocity_dispersion_2)}")
print(f"Velocity Dispersion for z = 1.6, combined Einstein radii: {format_to_nice_str(velocity_dispersion_combined)}")

Velocity Dispersion for z -> inf, error 0.1: (1.073e+03 +- 1.904e+00) km rad(1/2) / s
Velocity Dispersion for z -> inf, error 1.0: (1.07e+03 +- 1.90e+01) km rad(1/2) / s
Velocity Dispersion for z = 1.6, error 0.1: (1.276e+03 +- 2.264e+00) km rad(1/2) / s
Velocity Dispersion for z = 1.6, error 1.0: (1.28e+03 +- 2.26e+01) km rad(1/2) / s
Velocity Dispersion for z = 1.6, combined Einstein radii: (1.31e+03 +- 3.53e+01) km rad(1/2) / s


#### Mass calculation:

In [150]:
z_s = 1.6
H = QFloat(100, None, "km / (s Mpc)").to("1/s") # h = H_0/H

D_s_h = (2*c / H) * ((1+z_s)**(-1) - (1+z_s)**(-3/2)) # times h^-1
D_ds_h = (2*c / H) * (u * (1+z_s)**(-1) - (1+z_s)**(-3/2)) # times h^-1
D_d_h = (2*c / H) * ((1+z_d)**(-1) - (1+z_d)**(-3/2)) # times h^-1

enclosed_mass_h_1 = (c**2 / (4*G)) * (D_d_h*D_s_h / D_ds_h) * (einstein_radius_1.to("rad")**2)

enclosed_mass_h_2 = (c**2 / (4*G)) * (D_d_h*D_s_h / D_ds_h) * (einstein_radius_2.to("rad")**2)

enclosed_mass_h_combined = (c**2 / (4*G)) * (D_d_h*D_s_h / D_ds_h) * (einstein_radius_combined.to("rad")**2)

# print in solar masses:
mass_sun = QFloat(const.M_sun.value, None, const.M_sun.unit)
enclosed_mass_h_1 /= mass_sun
enclosed_mass_h_2 /= mass_sun
enclosed_mass_h_combined /= mass_sun


print(f"Enclosed Mass, error 0.1 arcsec: {format_to_nice_str(enclosed_mass_h_1)} M_sun*h^-1")
print(f"Enclosed Mass, error 1.0 arcsec: {format_to_nice_str(enclosed_mass_h_2)} M_sun*h^-1")

print(f"Enclosed Mass, combined Einstein radii: {format_to_nice_str(enclosed_mass_h_combined)} M_sun*h^-1")

# use fixed H_0:
H_0 = QFloat(67.66, 0.42, "km / (s Mpc)")
print(f"\nH_0 = {H_0.nominal}+-{H_0.uncertainty} {H_0.unit}")

h = (H_0.to("1/s")/H)
print(f"h = {h.nominal}")

print(f"Enclosed Mass, error 0.1 arcsec: {format_to_nice_str(enclosed_mass_h_1 * h**(-1))} M_sun")
print(f"Enclosed Mass, error 1.0 arcsec: {format_to_nice_str(enclosed_mass_h_2 * h**(-1))} M_sun")

print(f"Enclosed Mass, combined Einstein radii: {format_to_nice_str(enclosed_mass_h_combined * h**(-1))} M_sun*h^-1")


Enclosed Mass, error 0.1 arcsec: (1.063e+14 +- 7.548e+11) rad2 M_sun*h^-1
Enclosed Mass, error 1.0 arcsec: (1.06e+14 +- 7.55e+12) rad2 M_sun*h^-1
Enclosed Mass, combined Einstein radii: (1.2e+14 +- 1.3e+13) rad2 M_sun*h^-1

H_0 = 67.66+-0.42 km / (Mpc s)
h = 0.6766
Enclosed Mass, error 0.1 arcsec: (1.57e+14 +- 1.48e+12) rad2 M_sun
Enclosed Mass, error 1.0 arcsec: (1.6e+14 +- 1.1e+13) rad2 M_sun
Enclosed Mass, combined Einstein radii: (1.7e+14 +- 1.9e+13) rad2 M_sun*h^-1


#### Size calculation:

In [151]:
size_1 = (D_d_h.to("kpc") * einstein_radius_1.to("rad")) # times h^-1
size_2 = (D_d_h.to("kpc") * einstein_radius_2.to("rad")) # times h^-1
size_combined = (D_d_h.to("kpc") * einstein_radius_combined.to("rad")) # times h^-1

print(f"Size, error 0.1 arcsec: {size_1}*h^-1")
print(f"Size, error 1.0 arcsec: {size_2}*h^-1")
print(f"Size, combined Einstein radii: {size_combined}*h^-1")

Size, error 0.1 arcsec: 89.5+-0.3 kpc rad*h^-1
Size, error 1.0 arcsec: 89+-3 kpc rad*h^-1
Size, combined Einstein radii: 94+-5 kpc rad*h^-1


# P6

In [152]:
max_eff_core_radius = QFloat(0.45, None, None)

def get_max_core_radius(max_eff_radius, einstein_radius):
    max_core_radius = max_eff_radius * einstein_radius
    return max_core_radius

max_core_radius_1 = get_max_core_radius(max_eff_core_radius, einstein_radius_1)
max_core_radius_2 = get_max_core_radius(max_eff_core_radius, einstein_radius_2)
max_core_radius_combined = get_max_core_radius(max_eff_core_radius, einstein_radius_combined)

print(f"Upper core radius, error 0.1 arcsec: {max_core_radius_1}")
print(f"Upper core radius, error 1.0 arcsec: {max_core_radius_2}")
print(f"Upper core radius, combined Einstein radii: {max_core_radius_combined}")


Upper core radius, error 0.1 arcsec: 12.68+-0.05 arcsec
Upper core radius, error 1.0 arcsec: 12.7+-0.4 arcsec
Upper core radius, combined Einstein radii: 13.4+-0.7 arcsec
